## 재무정보(전체 재무제표) - 단일 회사(상장기업) - 반기 보고서
+ 작성: 임경호
+ 금융감독원 OPEN DART https://opendart.fss.or.kr/
+ 하나의 엑셀에 회사별로 각 시트에 저장
+ 회사별 비교표 구성 => 회사마다 데이터 입력을 표준적으로 하지 않아, 받아온 데이터로 상호 비교/매칭시킬 수 있지 않음.

### 1. 대상 회사/연도/보고서 설정 및 재무제표 정보 가져오기

In [1]:
import requests
import pandas as pd

fiscal_year = '2023'
listed_corp_codes = \
['01208885',    # 경동도시가스
 '00787376',    # 대성에너지
 '00128555',    # 삼천리
 '00130684']    # 서울도시가스
report_code = '11012'   # 반기보고서

In [2]:
# 재무제표 가져오기
url = 'https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json'
api_key = 'f2e08d4ed3de0ba3d5cbf59c04c223e02b1751a2'

dict_fi = {}

for corp_code in listed_corp_codes:
    params = {
        'corp_code': corp_code,
        'bsns_year': fiscal_year,
        'reprt_code': report_code,
        'fs_div': 'OFS',            # OFS:재무제표
        'crtfc_key': api_key,
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:             # URL GET 200 : 정상
        print("URL GET Error")
    if response.json()['status'] == '000':      # 데이터 유무 '000' : 정상
        # 데이터프레임 형태로 변환
        df_fi = pd.json_normalize(response.json()['list'])
        if not df_fi.empty:
            dict_fi[corp_code] = df_fi

In [60]:
for k, df in dict_fi.items():  
    print(k, ": ", len(df))

01208885 :  128
00787376 :  175
00128555 :  129
00130684 :  156


In [62]:
# 각 회사별로 가져온 데이터를 하나의 엑셀 파일에 각각의 시트로 생성
filename = ".".join([fiscal_year + '_반기보고서', "xlsx"])
path = "D:/PythonProject/data-gatherer/dart/data/" + filename
with pd.ExcelWriter(path) as writer:
    for k, df in dict_fi.items():
        df.to_excel(writer, sheet_name=k)

In [107]:
# # 각 회사의 계정명을 하나로 병합
# df_fi_all = pd.DataFrame()
# for k, df in dict_fi.items():    
#     df_fi_all = pd.concat([df_fi_all, df[['sj_div', 'account_nm', 'account_detail']]], ignore_index=True)

# df_fi_all.columns = ['sj_div', 'account_nm', 'account_detail']
# # account_id 중복 제거
# df_fi_all.drop_duplicates(['sj_div', 'account_nm', 'account_detail'], inplace=True)
# df_fi_all.insert(0, 'sj_div_account_nm', df_fi_all['sj_div'] + '_' + df_fi_all['account_nm'] + '_' + df_fi_all['account_detail'], allow_duplicates=False)
# df_fi_all.reset_index(inplace=True)     # 추후 Data Handling을 위해서는 반드시 reset_index 필수
# df_fi_all = df_fi_all.drop('index', axis=1)

In [106]:
# # 각 회사의 재무제표 합치기 - account_id 및 account_detail 기준으로 병합(어느 한쪽에라도 있으면 합친다)
# # 각 회사별로 가져온 데이터를 하나의 엑셀 파일에 각각의 시트로 생성
# filename = ".".join([fiscal_year + '_반기보고서(비교)', "xlsx"])
# path = "D:/PythonProject/data-gatherer/dart/data/" + filename
# with pd.ExcelWriter(path) as writer:
#     for k, df in dict_fi.items():
#         df_item = df[['sj_div', 'account_nm', 'account_detail', 'ord', 'thstrm_amount']]
#         df_item.insert(0, 'sj_div_account_nm', df_item['sj_div'] + '_' + df_item['account_nm'] + '_' + df_item['account_detail'], allow_duplicates=False)
#         df_item = df_item.drop('sj_div', axis=1)
#         df_item = df_item.drop('account_nm', axis=1)
#         df_item = df_item.drop('account_detail', axis=1)
#         df_item.columns = ['sj_div_account_nm', k+'_'+'ord', k+'_'+'thstrm_amount']
#         df_item.to_excel(writer, sheet_name=k)
#         # df_fi_all = pd.merge(df_fi_all, df_item, how='left', left_on='account_id_detail', right_on='account_id_detail')

In [110]:
# # 각 회사의 재무제표 합치기 - account_id 및 account_detail 기준으로 병합(어느 한쪽에라도 있으면 합친다)
# for k, df in dict_fi.items():
#     df_item = df[['sj_div', 'account_nm', 'account_detail', 'ord', 'thstrm_amount']]
#     df_item.insert(0, 'sj_div_account_nm', df_item['sj_div'] + '_' + df_item['account_nm'] + '_' + df_item['account_detail'], allow_duplicates=False)
#     df_item = df_item.drop('sj_div', axis=1)
#     df_item = df_item.drop('account_nm', axis=1)
#     df_item = df_item.drop('account_detail', axis=1)
#     df_item.columns = ['sj_div_account_nm', k+'_'+'ord', k+'_'+'thstrm_amount']
#     df_fi_all = pd.merge(df_fi_all, df_item, how='left', left_on='sj_div_account_nm', right_on='sj_div_account_nm')